In [206]:
import pandas as pd
import sqlalchemy
import json

In [207]:
cred = { 'user': 'vishnupriya', 'password': 'yFl#Cy3qzWW7f#OrxW4IB!P'}

def data_fetch_v2(query,*, 
                  cred=cred, db='aspiredb', 
                  host="13.235.168.161", **kwargs):
    """
    Fetches data for the given query from the database.

    Args:
        query (str):
        cred (dict): {"user" : "<user name>", "password" : "<password>"}
        db (str): 
        host (str):
        kwargs : 

    Returns:
        success_flag (bool):
        output (pd.DataFrame): 
        error_msg (str):
    """
    success_flag = False
    output = pd.DataFrame()
    error_msg = ""
    db_url = rf"mysql+pymysql://{cred['user']}:{cred['password']}@{host}/{db}"
    engine = sqlalchemy.create_engine(db_url)
    output = pd.read_sql(query, engine, **kwargs)
    print(output)
    success_flag = True

    try:
        db_url = rf"mysql+pymysql://{cred['user']}:{cred['password']}@{host}/{db}"
        engine = sqlalchemy.create_engine(db_url)
        output = pd.read_sql(query, engine, **kwargs)
        print(output)
        success_flag = True
    except Exception as e:
        success_flag = False
        error_msg = f"error in data_fetch, {type(e)._name_}: {str(e)}"
    finally:
        return success_flag, output, error_msg

In [209]:
data = """
select user_id, repayment_data from repayment_tranche_details where repayment_data is not null
"""
f1, repay, error = data_fetch_v2(data, host = "13.126.97.63")

                                      user_id  \
0        0072a6f6-7278-4527-8c7a-70760d3001e6   
1        0072a6f6-7278-4527-8c7a-70760d3001e6   
2        0072a6f6-7278-4527-8c7a-70760d3001e6   
3        0072a6f6-7278-4527-8c7a-70760d3001e6   
4        0072a6f6-7278-4527-8c7a-70760d3001e6   
...                                       ...   
1501200  39c0a5f8-e023-4e63-841b-1366e3dcff41   
1501201  39c0a5f8-e023-4e63-841b-1366e3dcff41   
1501202  3fa7e2ac-6f5f-4440-88be-0007d2cdee69   
1501203  3fa7e2ac-6f5f-4440-88be-0007d2cdee69   
1501204  346555b8-595f-4e16-b44f-270b966a5ecf   

                                            repayment_data  
0        {"data": [{"amount": 237.5, "repayment_id": "8...  
1        {"data": [{"amount": 196.75, "repayment_id": "...  
2        {"data": [{"amount": 49.25, "repayment_id": "8...  
3        {"data": [{"amount": 53.1, "repayment_id": "88...  
4        {"data": [{"amount": 63.4, "repayment_id": "88...  
...                                          

In [168]:
repay["repayment_data"][0]

'{"data": [{"amount": 237.5, "repayment_id": "88e7ed9b-5d0a-4e20-af29-227b64e2967c", "repayment_time": "2021-11-14 21:58:20"}]}'

In [165]:
repay.dtypes

user_id           object
repayment_data    object
dtype: object

In [166]:
js1 = json.loads(repay["repayment_data"])

TypeError: the JSON object must be str, bytes or bytearray, not Series

In [169]:
print(type(repay["repayment_data"]))  # Check if it is a Series or a single string


<class 'pandas.core.series.Series'>


In [210]:
repay["repayment_data"] = repay["repayment_data"].apply(json.loads)


In [211]:
repay["repayment_data_details"] = repay["repayment_data"].apply(lambda x: x["data"])


In [212]:
repay[["repayment_data_details"]]

,repayment_data_details
0,"[{'amount': 237.5, 'repayment_id': '88e7ed9b-5..."
1,"[{'amount': 196.75, 'repayment_id': '88e7ed9b-..."
2,"[{'amount': 49.25, 'repayment_id': '88e7ed9b-5..."
3,"[{'amount': 53.1, 'repayment_id': '88e7ed9b-5d..."
4,"[{'amount': 63.4, 'repayment_id': '88e7ed9b-5d..."
...,...
1501200,"[{'amount': 63.14, 'repayment_id': 'f82edbc3-8..."
1501201,"[{'amount': 2910, 'repayment_id': 'f82edbc3-8f..."
1501202,"[{'amount': 706.19, 'repayment_id': '8bef9995-..."
1501203,"[{'amount': 4000, 'repayment_id': '8bef9995-d4..."


In [ ]:
output of repay["repayment_data_details"][0] is [{"amount": 124.75, "repayment_id": "19f75a1b-14a3-4dfe-9e24-f9f1761dd9e6", "repayment_time": "2022-06-17 16:57:56"}, {"amount": 229.25, "repayment_id": "4d12d685-f3f3-4375-b9d2-ed8bdb0a564b", "repayment_time": "2022-07-05 11:05:01"}]

In [213]:
pd.json_normalize(repay["repayment_data_details"])

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,"{'amount': 237.5, 'repayment_id': '88e7ed9b-5d...",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,"{'amount': 196.75, 'repayment_id': '88e7ed9b-5...",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,"{'amount': 49.25, 'repayment_id': '88e7ed9b-5d...",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"{'amount': 53.1, 'repayment_id': '88e7ed9b-5d0...",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,"{'amount': 63.4, 'repayment_id': '88e7ed9b-5d0...","{'amount': 174.1, 'repayment_id': '8ec4e53d-5b...",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1501200,"{'amount': 63.14, 'repayment_id': 'f82edbc3-8f...",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1501201,"{'amount': 2910, 'repayment_id': 'f82edbc3-8fa...",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1501202,"{'amount': 706.19, 'repayment_id': '8bef9995-d...",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1501203,"{'amount': 4000, 'repayment_id': '8bef9995-d44...",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
repay["repayment_data"][0]["data"][0]["amount"]

repay["repayment_amount"] = repay["repayment_data"].apply(lambda x: x["data"][0]["amount"])
repay

,user_id,repayment_data,payment_date,repayment_amount
0,0072a6f6-7278-4527-8c7a-70760d3001e6,"{'data': [{'amount': 237.5, 'repayment_id': '8...",None,237.50
1,0072a6f6-7278-4527-8c7a-70760d3001e6,"{'data': [{'amount': 196.75, 'repayment_id': '...",None,196.75
2,0072a6f6-7278-4527-8c7a-70760d3001e6,"{'data': [{'amount': 49.25, 'repayment_id': '8...",None,49.25
3,0072a6f6-7278-4527-8c7a-70760d3001e6,"{'data': [{'amount': 53.1, 'repayment_id': '88...",None,53.10
4,0072a6f6-7278-4527-8c7a-70760d3001e6,"{'data': [{'amount': 63.4, 'repayment_id': '88...",None,63.40
...,...,...,...,...
1501200,39c0a5f8-e023-4e63-841b-1366e3dcff41,"{'data': [{'amount': 63.14, 'repayment_id': 'f...",None,63.14
1501201,39c0a5f8-e023-4e63-841b-1366e3dcff41,"{'data': [{'amount': 2910, 'repayment_id': 'f8...",None,2910.00
1501202,3fa7e2ac-6f5f-4440-88be-0007d2cdee69,"{'data': [{'amount': 706.19, 'repayment_id': '...",None,706.19
1501203,3fa7e2ac-6f5f-4440-88be-0007d2cdee69,"{'data': [{'amount': 4000, 'repayment_id': '8b...",None,4000.00


In [192]:

repay["repayment_data"]

0          {'data': [{'amount': 237.5, 'repayment_id': '8...
1          {'data': [{'amount': 196.75, 'repayment_id': '...
2          {'data': [{'amount': 49.25, 'repayment_id': '8...
3          {'data': [{'amount': 53.1, 'repayment_id': '88...
4          {'data': [{'amount': 63.4, 'repayment_id': '88...
                                 ...                        
1501200    {'data': [{'amount': 63.14, 'repayment_id': 'f...
1501201    {'data': [{'amount': 2910, 'repayment_id': 'f8...
1501202    {'data': [{'amount': 706.19, 'repayment_id': '...
1501203    {'data': [{'amount': 4000, 'repayment_id': '8b...
1501204    {'data': [{'amount': 25000, 'repayment_id': '2...
Name: repayment_data, Length: 1501205, dtype: object

In [ ]:
# final_data = []
# for k, v in list_ext.items():
#     j1 = json.loads(v)
#     for ke, va in j1.items():
#         j2 = json.loads(va)
#         for key, val in j2.items():
#             print(val)


# d1 = []
# for k, v in list_ext.items():
#     j1 = json.loads(v)
#     for key, val in j1.items():
#         d1.append(val)
# print(d1)



d1 = []
for k, v in list_ext.items():
    j1 = json.loads(v)
    for key, val in j1.items():
        d1.append(val)


In [ ]:
repay["data"] = repay["payment_data"].apply(lambda x: x["data"] if )

<bound method IndexOpsMixin.value_counts of 0          None
1          None
2          None
3          None
4          None
           ... 
1501200    None
1501201    None
1501202    None
1501203    None
1501204    None
Name: payment_date, Length: 1501205, dtype: object>

In [176]:
repay["payment_date"] = repay["repayment_data"]["data"]["repayment_time"]

KeyError: 'data'

In [175]:
repay["repayment_data"][0]

{'data': [{'amount': 237.5,
   'repayment_id': '88e7ed9b-5d0a-4e20-af29-227b64e2967c',
   'repayment_time': '2021-11-14 21:58:20'}]}

In [5]:
#1 create a dictinary, converting this to dict, orient as records
#2 check for i in and k, v in. 
#3 create a a variable where, records are shown without the column names (user_id and repaymnet data)
# they are k, v, using for loop and items() json load the values name them. call these json load as variables
# from the variable extract separately. 
# and then get the data in form user_id and jso_values. in rows.append()

In [109]:
repay

,user_id,repayment_data
0,0072a6f6-7278-4527-8c7a-70760d3001e6,"{""data"": [{""amount"": 237.5, ""repayment_id"": ""8..."
1,0072a6f6-7278-4527-8c7a-70760d3001e6,"{""data"": [{""amount"": 196.75, ""repayment_id"": ""..."
2,0072a6f6-7278-4527-8c7a-70760d3001e6,"{""data"": [{""amount"": 49.25, ""repayment_id"": ""8..."
3,0072a6f6-7278-4527-8c7a-70760d3001e6,"{""data"": [{""amount"": 53.1, ""repayment_id"": ""88..."
4,0072a6f6-7278-4527-8c7a-70760d3001e6,"{""data"": [{""amount"": 63.4, ""repayment_id"": ""88..."
...,...,...
1501200,39c0a5f8-e023-4e63-841b-1366e3dcff41,"{""data"": [{""amount"": 63.14, ""repayment_id"": ""f..."
1501201,39c0a5f8-e023-4e63-841b-1366e3dcff41,"{""data"": [{""amount"": 2910, ""repayment_id"": ""f8..."
1501202,3fa7e2ac-6f5f-4440-88be-0007d2cdee69,"{""data"": [{""amount"": 706.19, ""repayment_id"": ""..."
1501203,3fa7e2ac-6f5f-4440-88be-0007d2cdee69,"{""data"": [{""amount"": 4000, ""repayment_id"": ""8b..."


In [110]:
repay_dict = repay.to_dict(orient="records")


In [111]:
type(repay_dict)

list

In [112]:
repay_dict

[{'user_id': '0072a6f6-7278-4527-8c7a-70760d3001e6',
  'repayment_data': '{"data": [{"amount": 237.5, "repayment_id": "88e7ed9b-5d0a-4e20-af29-227b64e2967c", "repayment_time": "2021-11-14 21:58:20"}]}'},
 {'user_id': '0072a6f6-7278-4527-8c7a-70760d3001e6',
  'repayment_data': '{"data": [{"amount": 196.75, "repayment_id": "88e7ed9b-5d0a-4e20-af29-227b64e2967c", "repayment_time": "2021-11-14 21:58:20"}]}'},
 {'user_id': '0072a6f6-7278-4527-8c7a-70760d3001e6',
  'repayment_data': '{"data": [{"amount": 49.25, "repayment_id": "88e7ed9b-5d0a-4e20-af29-227b64e2967c", "repayment_time": "2021-11-14 21:58:20"}]}'},
 {'user_id': '0072a6f6-7278-4527-8c7a-70760d3001e6',
  'repayment_data': '{"data": [{"amount": 53.1, "repayment_id": "88e7ed9b-5d0a-4e20-af29-227b64e2967c", "repayment_time": "2021-11-14 21:58:20"}]}'},
 {'user_id': '0072a6f6-7278-4527-8c7a-70760d3001e6',
  'repayment_data': '{"data": [{"amount": 63.4, "repayment_id": "88e7ed9b-5d0a-4e20-af29-227b64e2967c", "repayment_time": "2021-11-

In [113]:
data = {a["user_id"]: a["repayment_data"] for a in repay_dict}

In [118]:
data

{'0072a6f6-7278-4527-8c7a-70760d3001e6': '{"data": [{"amount": 124.75, "repayment_id": "4d12d685-f3f3-4375-b9d2-ed8bdb0a564b", "repayment_time": "2022-07-05 11:05:01"}]}',
 '017e663d-a236-40d7-8743-de2aaa68d317': '{"data": [{"amount": 188.8, "repayment_id": "711b3180-b6a1-4170-b465-c97939e9bcd3", "repayment_time": "2021-06-05 20:47:24"}]}',
 '0226e5f7-5d0c-4f92-877c-a9fe91eec54b': '{"data": [{"amount": 25, "repayment_id": "04c19533-b594-44ac-9422-cc0af35c435f", "repayment_time": "2022-09-05 10:19:22"}]}',
 '03ec2296-69b3-48d5-8a33-9126fcd04794': '{"data": [{"amount": 318.6, "repayment_id": "98dae47c-f345-4b21-907c-aac6d67e5433", "repayment_time": "2022-04-02 13:22:03"}]}',
 '047f9cfd-2050-4353-a3ad-0326cec958f5': '{"data": [{"amount": 309.33, "repayment_id": "10679688-4cf7-49c9-ba1c-18f28e6a5634", "repayment_time": "2022-10-05 15:35:14"}]}',
 '050e63d4-89ee-4f80-bb57-12ec522823d1': '{"data": [{"amount": 17.5, "repayment_id": "a755791d-cee9-4d07-9068-1c75403f9384", "repayment_time": "20

In [137]:
row = []
for k,v in data.items():
    j1 = json.loads(v)
    row.append(j1['data'][0])

# result = []
# for value in data.values():
#     parsed_json = json.loads(value)  # Parse the JSON string
#     result.append(parsed_json['data'][0])

In [150]:
data

{'0072a6f6-7278-4527-8c7a-70760d3001e6': '{"data": [{"amount": 124.75, "repayment_id": "4d12d685-f3f3-4375-b9d2-ed8bdb0a564b", "repayment_time": "2022-07-05 11:05:01"}]}',
 '017e663d-a236-40d7-8743-de2aaa68d317': '{"data": [{"amount": 188.8, "repayment_id": "711b3180-b6a1-4170-b465-c97939e9bcd3", "repayment_time": "2021-06-05 20:47:24"}]}',
 '0226e5f7-5d0c-4f92-877c-a9fe91eec54b': '{"data": [{"amount": 25, "repayment_id": "04c19533-b594-44ac-9422-cc0af35c435f", "repayment_time": "2022-09-05 10:19:22"}]}',
 '03ec2296-69b3-48d5-8a33-9126fcd04794': '{"data": [{"amount": 318.6, "repayment_id": "98dae47c-f345-4b21-907c-aac6d67e5433", "repayment_time": "2022-04-02 13:22:03"}]}',
 '047f9cfd-2050-4353-a3ad-0326cec958f5': '{"data": [{"amount": 309.33, "repayment_id": "10679688-4cf7-49c9-ba1c-18f28e6a5634", "repayment_time": "2022-10-05 15:35:14"}]}',
 '050e63d4-89ee-4f80-bb57-12ec522823d1': '{"data": [{"amount": 17.5, "repayment_id": "a755791d-cee9-4d07-9068-1c75403f9384", "repayment_time": "20

In [154]:
final = []
for k,v in data.items():
    js1 = json.loads(v)
    for val in js1:
        final.append({'user_id':k,"date":val["data"][2]})


TypeError: string indices must be integers, not 'str'

In [156]:
final = []
for k,v in data.items():
    js1 = json.loads(v)
    for val in js1:
        print(val)


data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data
data


In [155]:
val

'data'

In [141]:
ouptut = {a["repayment_id"]: a['repayment_time'] for a in row}

In [149]:
ouptut

{'4d12d685-f3f3-4375-b9d2-ed8bdb0a564b': '2022-07-05 11:05:01',
 '711b3180-b6a1-4170-b465-c97939e9bcd3': '2021-06-05 20:47:24',
 '04c19533-b594-44ac-9422-cc0af35c435f': '2022-09-05 10:19:22',
 '98dae47c-f345-4b21-907c-aac6d67e5433': '2022-04-02 13:22:03',
 '10679688-4cf7-49c9-ba1c-18f28e6a5634': '2022-10-05 15:35:14',
 'a755791d-cee9-4d07-9068-1c75403f9384': '2021-06-02 23:15:32',
 '6232430f-e491-4062-b6e3-f821e5034ae4': '2021-10-02 16:10:19',
 '80f2faf1-59e2-4238-9d35-80029a4162cc': '2022-03-08 15:27:41',
 'c5075b95-3041-4c6a-ad14-b1935d94b106': '2022-01-07 19:17:14',
 '0a75486d-7702-4a4e-95f0-db1b57349aef': '2022-06-02 12:37:19',
 '25f2946b-ce8f-48b5-b1d5-9e494261da46': '2022-07-31 08:50:17',
 '54feb853-dcd9-4db2-b0ec-b1c34a3e09cf': '2022-10-11 16:23:06',
 'ef3b03d9-51de-4f96-ba31-b2f6cf68a241': '2021-08-08 12:53:22',
 '18e08ba6-65bd-419e-b061-758166bc1c03': '2021-12-06 17:42:22',
 '36e51051-832e-40fa-8712-6b9a3cfe837b': '2021-09-02 09:19:12',
 'fe3541b6-dc94-4545-bf51-81f2f90e1e2b':

In [151]:
final_result = []
for k, v in ouptut.items():
    final_result.append(v)

In [152]:
final_result

['2022-07-05 11:05:01',
 '2021-06-05 20:47:24',
 '2022-09-05 10:19:22',
 '2022-04-02 13:22:03',
 '2022-10-05 15:35:14',
 '2021-06-02 23:15:32',
 '2021-10-02 16:10:19',
 '2022-03-08 15:27:41',
 '2022-01-07 19:17:14',
 '2022-06-02 12:37:19',
 '2022-07-31 08:50:17',
 '2022-10-11 16:23:06',
 '2021-08-08 12:53:22',
 '2021-12-06 17:42:22',
 '2021-09-02 09:19:12',
 '2022-04-01 07:00:03',
 '2023-01-09 15:50:19',
 '2022-02-22 14:14:31',
 '2021-10-09 04:59:05',
 '2021-09-09 11:13:20',
 '2021-09-01 11:53:28',
 '2022-07-05 10:42:17',
 '2021-05-13 17:49:12',
 '2022-08-04 18:41:38',
 '2021-06-05 18:32:23',
 '2022-10-07 22:32:46',
 '2022-02-11 17:35:33',
 '2021-08-01 00:23:38',
 '2022-01-02 11:47:18',
 '2022-01-29 13:25:31',
 '2022-01-07 11:44:00',
 '2023-06-18 23:01:11',
 '2021-12-12 18:56:20',
 '2021-07-11 01:30:17',
 '2022-01-06 12:27:48',
 '2021-12-06 14:11:53',
 '2021-11-03 11:27:22',
 '2022-06-05 23:21:49',
 '2021-08-01 09:03:29',
 '2021-05-08 06:24:00',
 '2021-12-01 11:39:28',
 '2021-07-09 17: